In [0]:
from pyspark.sql import functions as F

In [0]:
spark.conf.set("spark.sql.adaptive.enabled", "false")
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

In [0]:
df = spark.range(1, 10000+1, 1, 5).select("id", F.rand(10).alias("value"))
df.display()

In [0]:
df.rdd.getNumPartitions()

In [0]:
df.write.mode("overwrite").format("parquet").saveAsTable("NonBucketedTable1")
df.write.mode("overwrite").format("parquet").saveAsTable("NonBucketedTable2")

In [0]:
df.write.bucketBy(5, "id").sortBy("id").mode("overwrite").format("parquet").saveAsTable("bucketedTable1")

In [0]:
df.write.bucketBy(5, "id").sortBy("id").mode("overwrite").format("parquet").saveAsTable("bucketedTable2")

In [0]:
bucket_df1 = spark.read.table("bucketedTable1")
bucket_df2 = spark.read.table("bucketedTable2")

In [0]:
bucket_df1.rdd.getNumPartitions()

In [0]:
bucket_df2.rdd.getNumPartitions()

In [0]:
joined_bucketed_df = (
    bucket_df1
    .join(
        bucket_df2,
        on="id",
        how="left"
    )
)
joined_bucketed_df.show()

In [0]:
non_bucketed_df1 = spark.table("NonBucketedTable1")
non_bucketed_df2 = spark.table("NonBucketedTable2")

In [0]:
joined_non_bucketed_df = (
    non_bucketed_df1
    .join(
        non_bucketed_df2,
        on="id",
        how="left"
    )
)
joined_non_bucketed_df.show()